In [2]:
import sys 
sys.path.append('../../')
from config.HIVESERVER2_token import *

import pendulum
from tqdm import trange
from jinja2 import Template
from pyutil.pysql.sql import SQLClient

from tensorflow.train import Example, Features, Feature, BytesList, Int64List, FloatList
from tensorflow.io import TFRecordWriter
import tensorflow as tf

client = SQLClient('mixx')

In [3]:
q = '''
SELECT
    a.*
    , c.feature AS res_rarity_feature
FROM (
    SELECT *
    FROM hive.ml_tmp_db.wgame_bunlde_sequence_sample
    WHERE dt = '{{ ds }}'
) AS a
INNER JOIN (
    SELECT acc_id, role_id, ARRAY_AGG(feature) AS feature
    FROM hive.ml_tmp_db.wgame_bundle_dynamic_pricing_feature_res_ratityv2
    WHERE dt = '{{ ds }}'
        AND feature_group in ('res_ratity','common')
    GROUP BY acc_id, role_id
) AS c
ON a.open_id = c.acc_id 
    AND a.user_id = c.role_id

'''

q = '''
SELECT a.*, b.feature AS role_feature
SELECT * FROM hive.ml_tmp_db.wgame_bunlde_sequence_sample as a
INNER JOIN (
    SELECT acc_id, role_id, ARRAY_AGG(feature) AS feature
    FROM hive.ml_tmp_db.wgame_bundle_dynamic_pricing_feature_res_ratityv2
    WHERE feature_group in ('res_ratity','common')
    GROUP BY acc_id, role_id
) b
ON open_id = acc_id AND user_id = role_id AND a.dt = b.dt
'''

temp = Template(q)

def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1e3
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1e6
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1e9
    else:
        return float(x)

def data2exmp(x):
    example = Example()
    f = example.features.feature

    f['bundle_price'].CopyFrom(Feature(bytes_list=BytesList(value=[x.get('bundle_price').encode()])))
    f['trigger_type'].CopyFrom(Feature(bytes_list=BytesList(value=[x.get('trigger_type').encode()])))
    f['max3pay'].CopyFrom(Feature(int64_list=Int64List(value=[x.get('max3pay')])))
    f['max7pay'].CopyFrom(Feature(int64_list=Int64List(value=[x.get('max7pay')])))
    f['label'].CopyFrom(Feature(int64_list=Int64List(value=[x.get('label')])))

    for name in ['gift_contents','last1bundle','last2bundle','last3bundle','last4bundle','last5bundle',]:
        value = x.get(name)
        if value is None:
            value = [0] * 23
        else: 
            value = eval(value)

            if len(value) <= 23:
                value = value + [0] * (23 - len(value))
            else:
                value = value[:23]

        f[name].CopyFrom(Feature(int64_list=Int64List(value=value)))

    for field in eval(x['res_rarity_feature']):
        for x in field.split('\x02'):
            name,value = x.split('\x03')
            if name == 'svr_region':
                f[name].CopyFrom(Feature(bytes_list=BytesList(value=[value.encode()])))
            else:
                f[name].CopyFrom(Feature(float_list=FloatList(value=[value_to_float(value)])))
    return example

DS = pendulum.Date(2023,6,17)
for i in trange(1,30):
    ds = DS.add(days=-i).to_date_string()
    client.reset()
    query = temp.render(ds = ds)
    client.execute_trino_query(query)
    data = client.to_pandas()
    with TFRecordWriter(f"../../data/tfrecord/0.0.2/{ds}.tfrecord") as writer:
        for x in data.to_dict(orient='records'):
            example = data2exmp(x)
            writer.write(example.SerializeToString())

100%|██████████| 29/29 [1:06:00<00:00, 136.58s/it]


In [16]:
dataset = tf.data.TFRecordDataset(f"../../data/tfrecord/0.0.2/2023-06-17.tfrecord")

In [18]:



feature_description = {
    'bundle_price': tf.io.FixedLenFeature([1], tf.string),
    'trigger_type' : tf.io.FixedLenFeature([1], tf.string),
    'max3pay' : tf.io.FixedLenFeature([1], tf.int64),
    'max7pay' : tf.io.FixedLenFeature([1], tf.int64),
    'label' : tf.io.FixedLenFeature([1], tf.int64),
    'gift_contents': tf.io.FixedLenFeature([23], tf.int64),
    'last1bundle': tf.io.FixedLenFeature([23], tf.int64),
    'last2bundle': tf.io.FixedLenFeature([23], tf.int64),
    'last3bundle': tf.io.FixedLenFeature([23], tf.int64),
    'last4bundle': tf.io.FixedLenFeature([23], tf.int64),
    'last5bundle': tf.io.FixedLenFeature([23], tf.int64),
    # 'gift_contents': tf.io.VarLenFeature(tf.int64),
    # 'last1bundle': tf.io.VarLenFeature(tf.int64),
    # 'last2bundle': tf.io.VarLenFeature(tf.int64),
    # 'last3bundle': tf.io.VarLenFeature(tf.int64),
    # 'last4bundle': tf.io.VarLenFeature(tf.int64),
    # 'last5bundle': tf.io.VarLenFeature(tf.int64),

    'svr_region': tf.io.FixedLenFeature([1], tf.string),
    "acc_first_purchase_price_usd": tf.io.FixedLenFeature([1], tf.float32),
    "acc_role_cnt": tf.io.FixedLenFeature([1], tf.float32),
    "acc_total_charge_cnt": tf.io.FixedLenFeature([1], tf.float32),
    "acc_total_charge_sum_usd": tf.io.FixedLenFeature([1], tf.float32),
    "acc_total_refund_cnt": tf.io.FixedLenFeature([1], tf.float32),
    "acc_total_refund_sum_usd": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_count_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_count_last_day_campaign_stage_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_count_last_day_tech_unlock_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_count_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_count_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_price_campaign_stage_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_price_tech_unlock_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_sum_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_sum_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_buy_sum_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_campaign_stage_trigger_9999": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_last_day_campaign_stage_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_last_day_tech_unlock_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "bundle_pop_count_tech_unlock_trigger_9999": tf.io.FixedLenFeature([1], tf.float32),
    "buy_rate_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "buy_rate_last_day_campaign_stage_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "buy_rate_last_day_tech_unlock_trigger": tf.io.FixedLenFeature([1], tf.float32),
    "buy_rate_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "if_accounts_first_role": tf.io.FixedLenFeature([1], tf.float32),
    "if_pruchase_active_in_1day": tf.io.FixedLenFeature([1], tf.float32),
    "if_pruchase_active_in_30day": tf.io.FixedLenFeature([1], tf.float32),
    "if_pruchase_active_in_3day": tf.io.FixedLenFeature([1], tf.float32),
    "if_pruchase_active_in_7day": tf.io.FixedLenFeature([1], tf.float32),
    "is_acc_transfer_from_old": tf.io.FixedLenFeature([1], tf.float32),
    "is_acc_transfer_to_new": tf.io.FixedLenFeature([1], tf.float32),
    "is_role_transfer_from_old": tf.io.FixedLenFeature([1], tf.float32),
    "is_role_transfer_to_new": tf.io.FixedLenFeature([1], tf.float32),
    "is_valid": tf.io.FixedLenFeature([1], tf.float32),
    "level": tf.io.FixedLenFeature([1], tf.float32),
    "max_role_level": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_in_cout_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_in_cout_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_in_cout_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_out_cout_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_out_cout_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "money_flow_out_cout_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_1499_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_1499_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_1999_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_1999_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_2999_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_2999_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_4999_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_4999_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_499_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_499_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_9999_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_9999_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_999_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_999_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max_last_3days": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max_last_month": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_max_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_last_3days": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_last_day": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_last_month": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_last_week": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_last_week_adjust": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_max_ratio": tf.io.FixedLenFeature([1], tf.float32),
    "purchase_price_sum_max_ratio_com2": tf.io.FixedLenFeature([1], tf.float32),
    "role_create_os_android": tf.io.FixedLenFeature([1], tf.float32),
    "role_create_os_ios": tf.io.FixedLenFeature([1], tf.float32),
    "role_first_purchase_price_usd": tf.io.FixedLenFeature([1], tf.float32),
    "role_last_login_duration": tf.io.FixedLenFeature([1], tf.float32),
    "role_last_purchase_price_usd": tf.io.FixedLenFeature([1], tf.float32),
    "role_lifespan": tf.io.FixedLenFeature([1], tf.float32),
    "role_login_last_now": tf.io.FixedLenFeature([1], tf.float32),
    "role_login_secondlast_last": tf.io.FixedLenFeature([1], tf.float32),
    "role_login_secondlast_now": tf.io.FixedLenFeature([1], tf.float32),
    "role_logout_last_now": tf.io.FixedLenFeature([1], tf.float32),
    "role_no": tf.io.FixedLenFeature([1], tf.float32),
    "role_purchase_first_last": tf.io.FixedLenFeature([1], tf.float32),
    "role_purchase_first_now": tf.io.FixedLenFeature([1], tf.float32),
    "role_purchase_last_now": tf.io.FixedLenFeature([1], tf.float32),
    "role_total_charge_cnt": tf.io.FixedLenFeature([1], tf.float32),
    "role_total_charge_sum_usd": tf.io.FixedLenFeature([1], tf.float32),
    "role_total_refund_cnt": tf.io.FixedLenFeature([1], tf.float32),
    "role_total_refund_sum_usd": tf.io.FixedLenFeature([1], tf.float32),
    "role_type_fraud": tf.io.FixedLenFeature([1], tf.float32),
    "role_type_no_install_info": tf.io.FixedLenFeature([1], tf.float32),
    "vip_level": tf.io.FixedLenFeature([1], tf.float32),


    # 'research_building_level': tf.io.FixedLenFeature([1], tf.float32),
    # 'city_level': tf.io.FixedLenFeature([1], tf.float32),
    # 'chapter_task': tf.io.FixedLenFeature([1], tf.float32),

    'resource_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'speedup_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'top_key_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'officer_bar_ticket_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'common_equipment_exp_consume_levelup_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'common_equipment_exp_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'common_equipment_exp_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_equipment_exp_consume_levelup_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_equipment_exp_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_equipment_exp_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'ammunition_consume_levelup_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'ammunition_consume_levelup_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'ammunition_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'ammunition_consume_blueprint_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'ammunition_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_ammunition_consume_levelup_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_ammunition_consume_levelup_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_ammunition_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_ammunition_consume_blueprint_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_ammunition_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'elements_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'elements_consume_blueprint_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'elements_consume_reform_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'elements_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_elements_consume_blueprint_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_elements_consume_blueprint_10_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_elements_consume_reform_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'airforce_elements_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'replacement_wrench_consume_mechanic_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'replacement_wrench_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'officer_exp_consume_lvlup_1_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'officer_exp_rarity': tf.io.FixedLenFeature([1], tf.float32),
    'component_consume_diff_num_relatively': tf.io.FixedLenFeature([1], tf.float32),
    'chest_choice_replacement_rarity': tf.io.FixedLenFeature([1], tf.float32),
}


def decode_fn(x):
    e = tf.io.parse_single_example(x, feature_description)
    max3pay = e.pop('max3pay')
    max7pay = e.pop('max7pay')
    label = e.pop('label')
    bundle_price = e.pop('bundle_price')
    return (e, (max3pay, max7pay, label, bundle_price))

d = dataset.map(decode_fn).batch(32)
x = next(iter(d))

x

In [42]:
feature_name = [
    "acc_first_purchase_price_usd",
    "acc_role_cnt",
    "acc_total_charge_cnt",
    "acc_total_charge_sum_usd",
    "acc_total_refund_cnt",
    "acc_total_refund_sum_usd",
    "bundle_pop_buy_count_last_day",
    "bundle_pop_buy_count_last_day_campaign_stage_trigger",
    "bundle_pop_buy_count_last_day_tech_unlock_trigger",
    "bundle_pop_buy_count_last_week",
    "bundle_pop_buy_count_last_week_adjust",
    "bundle_pop_buy_max_last_day",
    "bundle_pop_buy_max_last_week",
    "bundle_pop_buy_max_last_week_adjust",
    "bundle_pop_buy_max_price_campaign_stage_trigger",
    "bundle_pop_buy_max_price_campaign_stage_trigger_commax",
    "bundle_pop_buy_max_price_tech_unlock_trigger",
    "bundle_pop_buy_max_price_tech_unlock_trigger_commax",
    "bundle_pop_buy_sum_last_day",
    "bundle_pop_buy_sum_last_week",
    "bundle_pop_buy_sum_last_week_adjust",
    "bundle_pop_count_campaign_stage_trigger_9999",
    "bundle_pop_count_last_day",
    "bundle_pop_count_last_day_campaign_stage_trigger",
    "bundle_pop_count_last_day_tech_unlock_trigger",
    "bundle_pop_count_last_week",
    "bundle_pop_count_last_week_adjust",
    "bundle_pop_count_tech_unlock_trigger_9999",
    "buy_rate_last_day",
    "buy_rate_last_day_campaign_stage_trigger",
    "buy_rate_last_day_tech_unlock_trigger",
    "buy_rate_last_week",
    "level",
    "max_role_level",
    "money_flow_in_cout_last_day",
    "money_flow_in_cout_last_week",
    "money_flow_in_cout_last_week_adjust",
    "money_flow_out_cout_last_day",
    "money_flow_out_cout_last_week",
    "money_flow_out_cout_last_week_adjust",
    "purchase_price_1499_last_week",
    "purchase_price_1499_last_week_adjust",
    "purchase_price_1999_last_week",
    "purchase_price_1999_last_week_adjust",
    "purchase_price_2999_last_week",
    "purchase_price_2999_last_week_adjust",
    "purchase_price_4999_last_week",
    "purchase_price_4999_last_week_adjust",
    "purchase_price_499_last_week",
    "purchase_price_499_last_week_adjust",
    "purchase_price_9999_last_week",
    "purchase_price_9999_last_week_adjust",
    "purchase_price_999_last_week",
    "purchase_price_999_last_week_adjust",
    "purchase_price_max",
    "purchase_price_max_last_3days",
    "purchase_price_max_last_day",
    "purchase_price_max_last_month",
    "purchase_price_max_last_week",
    "purchase_price_max_last_week_adjust",
    "purchase_price_sum_last_3days",
    "purchase_price_sum_last_day",
    "purchase_price_sum_last_month",
    "purchase_price_sum_last_week",
    "purchase_price_sum_last_week_adjust",
    "purchase_price_sum_max_ratio",
    "purchase_price_sum_max_ratio_com2",
    "role_create_os_android",
    "role_create_os_ios",
    "role_first_purchase_price_usd",
    "role_last_login_duration",
    "role_last_purchase_price_usd",
    "role_lifespan",
    "role_login_last_now",
    "role_login_secondlast_last",
    "role_login_secondlast_now",
    "role_logout_last_now",
    "role_no",
    "role_purchase_first_last",
    "role_purchase_first_now",
    "role_purchase_last_now",
    "role_total_charge_cnt",
    "role_total_charge_sum_usd",
    "role_total_refund_cnt",
    "role_total_refund_sum_usd",
    "vip_level",
]



features {
  feature {
    key: "acc_first_purchase_price_usd"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "acc_role_cnt"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "acc_total_charge_cnt"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "acc_total_charge_sum_usd"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "acc_total_refund_cnt"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "acc_total_refund_sum_usd"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "airforce_ammunition_consume_blueprint_10_diff_num_relatively"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feature {
    key: "airforce_ammunition_consume_blueprint_1_diff_num_relatively"
    value {
      float_list {
        value: 0.0
      }
    }
  }
  feat